In [1]:
import polars as pl

import rustystats as rs


data = pl.read_parquet("https://raw.githubusercontent.com/PricingFrontier/pricing-data-example/917c853e256df8d5814721ab56f72889a908bb08/data/processed/frequency_set.parquet")

In [2]:
model = rs.glm(
    formula="ClaimCount ~ VehPower + VehAge + C(Area) + C(Region)",
    data=data,
    family="poisson"
).fit()

In [3]:
print(model.summary())

                                 GLM Results                                  

Family:              Poisson         No. Observations:        678012
Link Function:       (default)       Df Residuals:            677984
Method:              IRLS            Df Model:                    27
Scale:               0.3138          Iterations:                   6

Log-Likelihood:         -140874.1999 Deviance:                212748.2537
AIC:                     281804.3999 Null Deviance:           214041.4441
BIC:                     282124.3537 Pearson chi2:              716467.57
Converged:           True           

------------------------------------------------------------------------------
Variable           Coef    Std.Err        z    P>|z|                 95% CI     
------------------------------------------------------------------------------
Intercept       -2.7969     0.0505  -55.362  <0.0001   [ -2.8960,  -2.6979]  ***
VehPower        -0.0085     0.0015   -5.790  <0.0001   [ -0.011

In [ ]:
           # Full regression table
print(result.coef_table())        # As pandas DataFrame
print(result.relativities())      # exp(coef) for log-link